Original Code

In [3]:
#Import the needed modules
import pya
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Tester_Pattern.gds', mesh:str = 'Test_Mesh_Small.csv', target_layer:int = 0, LL:list=(0,0), UR:list=(382, 490),output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Import the .gds
    gds = pya.Layout()
    gds.read(gds_file)
    print('File read...')

    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Hard coding layer levels (this can be improved in the future...)
    bias_lyr_index = 42
    final_lyr_index = 43

    #Create bias layer for eventual use
    bias_lyr = gds.layer(bias_lyr_index,0,'Initial Biasing Layer')

    #Create final bias layer
    final_lyr = gds.layer(final_lyr_index,0,'Final Biased Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print('Mesh read...')

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells)
    top_cell = gds.top_cell()
    print('Top cell chosen, entering loop...')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in tqdm(columns):
                #Define the edges of the mesh at each location
                l_edge = LL[0]+mesh_x*(j-1)
                b_edge = UR[1]-mesh_y*i
                r_edge = LL[0]+mesh_x*j
                t_edge = UR[1]-mesh_y*(i-1)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]

                #Takes polygons from the copy layer in a given search region to keep in the "touching" pya.Region 
                search_region = pya.DBox(l_edge,b_edge,r_edge,t_edge)
                touching = pya.Region(top_cell.begin_shapes_rec_touching(target_layer, search_region))

                #Biases "touching" and copies it into the top cell in the bias layer
                touch_up = touching.sized(int(biasing))
                top_cell.shapes(bias_lyr).insert(touch_up)
                
    
    print("Finished Mesh Loop")

    #Merges any overlapping polygons and places the final biased data onto the "final" layer
    bias_merge = pya.Region(top_cell.begin_shapes_rec_touching(bias_lyr,pya.DBox(LL[0],LL[1],UR[0],UR[1])))
    bias_merge.merge()
    top_cell.shapes(final_lyr).insert(bias_merge)

    #Removes unneeded bias layer from the layout
    gds.clear_layer(bias_lyr)
    

    #Writes the new file
    return gds.write(output_file)




rpdb()



File read...
Mesh read...
There are 21 columns in the mesh
There are 21 rows in the mesh
Mesh X is 19.1nm
Mesh Y is 24.5nm
Top cell chosen, entering loop...


100%|██████████| 20/20 [00:02<00:00,  9.31it/s]

Finished Mesh Loop


New Test (trying with clips)

In [48]:
#Import the needed modules
import pya
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Tester_Pattern.gds', mesh:str = 'Test_Mesh_Tiny.csv', target_layer:int = 0, LL:list=(-1,-.5), UR:list=(383, 492),output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Import the .gds
    gds = pya.Layout()
    gds.read(gds_file)
    print('File read...')

    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Hard coding layer levels (this can be improved in the future...)
    bias_lyr_index = 42
    final_lyr_index = 43

    #Create bias layer for eventual use
    bias_lyr = gds.layer(bias_lyr_index,0,'Initial Biasing Layer')

    #Create final bias layer
    final_lyr = gds.layer(final_lyr_index,0,'Final Biased Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print('Mesh read...')

    #Determine maximum absolute bias applied, in order to size up the cell regions
    mesh_max = pdb.max().max()
    mesh_min = pdb.min().min()
    if abs(mesh_max) >= abs(mesh_min):
         bias_max = mesh_max
    else:
         bias_max = abs(mesh_min)

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells)
    top_cell = gds.top_cell()
    print('Top cell chosen, entering loop...')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in tqdm(columns):
                #Define the edges of the mesh at each location
                l_edge = LL[0]+(mesh_x*(j-1)-bias_max)
                b_edge = UR[1]-(mesh_y*i+bias_max)
                r_edge = LL[0]+(mesh_x*j+bias_max)
                t_edge = UR[1]-(mesh_y*(i-1)-bias_max)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]

                #Clips mesh area from the polygon, preserving hierarchy, and names it
                clip = gds.clip(top_cell,pya.DBox(l_edge,b_edge,r_edge,t_edge))
                clip.name= f'Bias_Cell_{i}_{j}'

                #Loop to identify each shape and size it up by the bias value amount
                
                
    
    print("Finished Mesh Loop")
    

    #Writes the new file
    return gds.write(output_file)




rpdb()



File read...
Mesh read...
There are 3 columns in the mesh
There are 3 rows in the mesh
Mesh X is 192.0nm
Mesh Y is 246.25nm
Top cell chosen, entering loop...


  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: No overload with matching arguments in Cell.insert